<a href="https://colab.research.google.com/github/iesous-kurios/powerBi/blob/master/PowerBI_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
#@title
# import libraries which provide math functions to our data imports
import statistics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xlwt 
from xlwt import Workbook
import seaborn as sns
import datetime
from datetime import date, timedelta
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import style

sns.__version__

'0.10.0'

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#@title
# import weekly data from yearly FPS folder
df = pd.read_excel('/content/drive/My Drive/FPData/YTD Data/ClientTrackExport (5).xlsx')
#df = pd.read_excel ('~/environments/fp_env/family_promise_data/data/ClientTrackExport (5).xlsx')

In [0]:
# get list of currently enrolled guests (guests that don't have an exit date)
currently_enrolled = df[df['3.11 Exit Date'].isnull()]

In [16]:
currently_enrolled['3.3 Birthdate'].value_counts()

2018-01-01    2
2008-10-12    1
2008-04-03    1
2007-06-22    1
1981-07-04    1
             ..
1980-01-01    1
1988-03-04    1
2018-04-09    1
1993-01-01    1
2018-12-21    1
Name: 3.3 Birthdate, Length: 130, dtype: int64

In [0]:
bdays_date = (date.today()-timedelta(days=15)).isoformat()
current_bdays = currently_enrolled[currently_enrolled['Date of Last ES Stay (Beta)'] >= bdays_date ]

In [0]:
from datetime import date

def calculate_age(born):
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

In [40]:
# Python3 code to  calculate age in years 
from datetime import date 
  
def turning(born): 
    today = date.today() 
    this_month = date.today().month
    this_day = date.today().day
    try:  
        birthday = born.replace(year = today.year) 
        birthmonth = born.replace(month = today.month)
        birth_day = born.replace(day = today.day)
  
    # raised when birth date is February 29 
    # and the current year is not a leap year 
    except ValueError:  
        birthday = born.replace(year = today.year, 
                  month = born.month + 1, day = 1) 
  
    if birthday > today:
        if birthmonth == this_month:
            if birth_day > this_day:
                return today.year - born.year + 1
            if birth_day <= this_day:
                return today.year - born.year 
        else:
            return today.year - born.year        

    else: 
        return today.year - born.year 
          
# Driver code 
print(turning(date(1997, 2, 3)), "years") 

23 years


In [41]:
current_bdays['age'] = current_bdays['3.3 Birthdate'].apply(calculate_age)
current_bdays['age2'] = current_bdays['3.3 Birthdate'].apply(calculate_age)
current_bdays['turning'] = current_bdays['3.3 Birthdate'].apply(turning)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [0]:
bday_df = current_bdays[['3.1 FirstName', '3.1 LastName', '3.3 Birthdate','turning']].copy()

In [0]:

currentMonth = date.today().month
currentMonth
two_months = date.today().month + 2

last_month = date.today().month - 1

In [0]:
bday_df['month'] = pd.DatetimeIndex(bday_df['3.3 Birthdate']).month
bday_df['day'] = pd.DatetimeIndex(bday_df['3.3 Birthdate']).day

In [0]:
upcomming_bdays = bday_df[(bday_df['month'] > last_month) & (bday_df['month'] < two_months) ]

In [0]:
upcomming_bdays.to_excel('/content/drive/My Drive/FPData.close_bdays.xlsx')

In [0]:
upcomming_bdays

In [0]:
# create new list of those currently enrolled who haven't stayed with us for
# two weeks or more



current_date = date.today().isoformat()   
days_before = (date.today()-timedelta(days=30)).isoformat()

need_to_exit = currently_enrolled[currently_enrolled['Date of Last ES Stay (Beta)'] < days_before ]

In [0]:
exit_df = need_to_exit[['3.1 FirstName', '3.1 LastName', '5.8 Personal ID', '5.9 Household ID', '3.15 Relationship to HoH', 'CaseMembers', '3.10 Enroll Date', 'Date of Last ES Stay (Beta)']].copy()

In [0]:
#exit_df.head()

In [0]:
exit_df.to_excel('/content/drive/My Drive/FPData/YTD Data/YTDexit_df.xlsx')

In [0]:
households = df['5.9 Household ID'].unique()
len(households)

In [0]:
# print total number of families served (unduplicated)
unique_households = df['5.9 Household ID'].nunique()
unique_households

In [0]:
#@title
# replace zeros in data with "Nan" to calculate more accurate average bednights 
df['Count of Bed Nights (Housing Check-ins)'] = df['Count of Bed Nights (Housing Check-ins)'].replace(0, np.NaN)

In [0]:
avg_bn = df['Count of Bed Nights (Housing Check-ins)'].mean()
avg_bn = round(avg_bn, 2)
avg_bn

In [0]:
#@title

exit_reasons = ['Rental by client with RRH or equivalent subsidy', 
                'Rental by client, no ongoing housing subsidy', 
                'Staying or living with family, permanent tenure', 
                'Rental by client, other ongoing housing subsidy',
                'Permanent housing (other than RRH) for formerly homeless persons', 
                'Staying or living with friends, permanent tenure', 
                'Owned by client, with ongoing housing subsidy', 
                'Staying or living with friends, permanent tenure', 
                'Rental by client, VASH housing Subsidy'
               ]

temp_reasons = ['Staying or living with family, temporary tenure (e.g., room, apartment or house)', 
               'Staying or living with friends, temporary tenure (e.g., room, apartment or house)', 
                
               ]

In [0]:
# drop rows with no exit destination (current guests at time of report)
df = df.dropna(subset=['3.12 Exit Destination'])

In [0]:
 # create target column (multiple types of exits to perm)
df['perm_leaver'] = df['3.12 Exit Destination'].isin(exit_reasons)

In [0]:
#df.head()

In [0]:
perm_leaver = df[df['perm_leaver'] == True]

In [0]:
total_perm = perm_leaver.perm_leaver.value_counts().sum()

In [0]:
total_perm

In [0]:
non_perm = df[df['perm_leaver'] == False]

In [0]:
total_non_perm = non_perm.perm_leaver.value_counts().sum()

In [0]:
total_non_perm

In [0]:
perm_percent = total_perm / total_non_perm

In [0]:
perm_percent = round(perm_percent, 2)*100
perm_percent

In [0]:
df['3.12 Exit Destination'].value_counts()

In [0]:
unknowns = ['Client refused', 'No exit interview completed']

In [0]:
unknown_df = df[df['3.12 Exit Destination'].isin(unknowns)]

In [0]:
unknown_exits = unknown_df['3.12 Exit Destination'].value_counts().sum()

In [0]:
unknown_exits

In [0]:
unknown_percent = unknown_exits / total_non_perm
unknown_percent = round(unknown_percent, 2) *100
unknown_percent

In [0]:
exits = df['3.12 Exit Destination'].value_counts().sum()
exits

In [0]:
exits = df['3.12 Exit Destination'].value_counts()

In [0]:
# take sum of ES and Trans exits and divide by total exits, then multiply by 100 to get percentage
esExits = exits['Emergency shelter, including hotel or motel paid for with emergency shelter voucher, or RHY-funded Host Home shelter'] 
#transExits = ((exits['Transitional Housing for homeless persons (including homeless youth)'] / exits.sum()) * 100)
# round the answer to nearest hundreth place
esExits = (round(esExits, 2))
#transExits = (round(transExits, 2))
esExits
#print('Exits to Transitional',transExits,'%')

In [0]:
tempExits = df['3.12 Exit Destination'].isin(temp_reasons).sum()

tempExits

In [0]:
transitional = df['3.12 Exit Destination'] == 'Transitional Housing for homeless persons (including homeless youth)'
transitional = transitional.sum()
transitional

In [0]:
exits.sum()

In [0]:
total_guests = total_non_perm + total_perm

In [0]:
trans_percent = (transitional / exits.sum())*100
trans_percent = round(trans_percent, 2)
trans_percent

In [0]:
#zips_master

In [0]:
# export zips into an excel sheet for Power BI visual
zips_master = pd.read_excel('/content/drive/My Drive/FPData/uszips.xlsx')



# Import pandas library 
import pandas as pd 
  
# initialize list of lists 
zips_bi = df['V5 Zip'].value_counts().to_frame('Total').reset_index(level=0, inplace=False)
zips_bi = zips_bi.rename(columns={"index": "V5 Zip"})
zips_bi = zips_bi.merge(zips_master,on='V5 Zip')
zips_bi['V5 City'].value_counts()

In [0]:
type(zips_bi)

In [0]:
zips_bi.to_excel('/content/drive/My Drive/FPData/YTD Data/weeklyYTDzips.xlsx', header=True, index=False)

In [0]:
other_reasons = ['Place not meant for habitation (e.g., a vehicle, an abandoned building, bus/train/subway station/airport or anywhere outside)',
                 'Hotel or Motel paid for without Emergency Shelter Voucher', 'Other', 'Substance Abuse Treatment or Detox Center',
                 'Foster Care Home or Foster Care Group Home', 'Psychiatric Hospital or Other Psychiatric Facility']

In [0]:
other = df['3.12 Exit Destination'].isin(other_reasons)

In [0]:
other.sum()
other = other.sum()
other

In [0]:
df['4.04.7 Private - Employer']

In [0]:
df['3.12 Exit Destination'].value_counts()

In [0]:
print(total_guests, 'total guests served YTD')
print(total_non_perm, 'total non perm leavers')
print(total_perm, 'total perm leavers')
print(esExits, 'total ES leavers')
print(tempExits, 'total temp leavers')
print(transitional, 'total trans leavers')
print(other, 'total leavers to "other"')
print(unknown_exits, 'total unknown leavers')
print('sum of all groups to make sure it matches total served:',total_perm+esExits+tempExits+transitional+other+unknown_exits)

In [0]:
assert((total_perm+esExits+tempExits+transitional+other+unknown_exits) == (total_non_perm+total_perm))

In [0]:
perm_bi = '{} %'.format(perm_percent)
es_bi = '{} %'.format(esExits)
temp_bi = '{} %'.format(tempExits)
trans_bi = '{} %'.format(trans_percent)
unknown_bi = '{} %'.format(unknown_percent)

In [0]:
power_bi = pd.DataFrame({'Permanent Leavers': total_perm, 'Emergency Shelter Leavers': esExits, 
                         'Temporary Leavers': tempExits, 'Transitional Leavers': transitional, 
                         'Unknown Leavers': unknown_exits, 'Other Leavers': other, 'Total': total_guests }, index=[0])


In [0]:
# Import pandas library 
import pandas as pd 
  
# initialize list of lists 
data = [['Permanent Leavers', total_perm], ['Emergency Shelter Leavers', esExits], 
        ['Temporary Leavers', tempExits], ['Transitional Leavers', transitional],
        ['Unknown Leavers', unknown_exits], ['Other Leavers', other]] 
  


# Create the pandas DataFrame 
power_bi = pd.DataFrame(data, columns = ['Exit Destination', 'Total']) 
  
# print dataframe. 

In [0]:
power_bi

In [0]:
power_bi.Total.sum()

In [0]:
power_bi.to_excel('/content/drive/My Drive/FPData/YTD Data/YTDweeklyDataNew.xlsx')

In [0]:
for col in df:
    print(col)

In [0]:
df['3.12 Exit Destination'].value_counts()